In [ ]:
import numpy as np
import pandas as pd
SUM_M = pd.DataFrame()

<center><h3> Capacities analysis for manufacturing facilities planning</h3></center>

workstation_id  | Process  | Efficiency (%)
------------- | ------------- | -------------
1  | Weighing  | 100
2  | Peeling  | 85
3  | Slicing  | 98 
4  | Fry  | Variable 

Let's consider some output flowing straight to final product warehouse, would say 1000 kg per year:

In [2]:
pair_ws_perf = []
nj = float(input("Insert the quantity of raw material expected process in '(kg/year)': "))
n_ws = int(input("Insert the quantity of work steps for this product: "))
for i in range(n_ws):
    pair = [x for x in input("Insert the work station-performance pair '(i. e:  2 0.69/zn)': ").split()]
    pair_ws_perf.append(pair)
pair_ws_perf = np.array(pair_ws_perf)

Insert the quantity of raw material expected process in '(kg/year)': 1000
Insert the quantity of work steps for this product: 4
Insert the work station-performance pair '(i. e:  2 0.69/zn)': 1 1
Insert the work station-performance pair '(i. e:  2 0.69/zn)': 2 0.85
Insert the work station-performance pair '(i. e:  2 0.69/zn)': 3 0.98
Insert the work station-performance pair '(i. e:  2 0.69/zn)': 4 z2


In [3]:
input_ws = []
for idx, value in enumerate(pair_ws_perf[::-1,1]):
    try:
        if idx == 0:
            try:
                val = nj/float(value)
                input_ws.append(val)
            except:
                input_ws.append(str(value))
        else:
            if isinstance(input_ws[len(input_ws)-1], str) == False:
                val = val/float(value)
                input_ws.append(val)
            else:
                if value[0] == 'z':
                    input_ws.append(str(value))
                else:
                    input_ws.append(str(value))
                    
    except:
        input_ws.append(str(value)) 

In [4]:
for k, f in zip(input_ws, pair_ws_perf[::-1,0]):
    print("entry value 'in (kg/year)' work station pair: {0} {1} ".format(k, f))

entry value 'in (kg/year)' work station pair: z2 4 
entry value 'in (kg/year)' work station pair: 0.98 3 
entry value 'in (kg/year)' work station pair: 0.85 2 
entry value 'in (kg/year)' work station pair: 1 1 


In [5]:
input_ws

['z2', '0.98', '0.85', '1']

The function just serves up workstation_id and their corresponding performance as it's arguments and returns a pd.DataFrame() object that represents the flow intensity matrix, for calling up this function try:
```
material_balance(input_ws, pair_ws_perf)
```

In [6]:
def material_balance(input_workstations, pair_ws_performance):
    
    global SUM_M
    
    if all(isinstance(x, float) for x in input_workstations) == True:
        m_labels= set(list(pair_ws_performance[::-1,0]) +['Alm_MP', 'Alm_PT'])
        M = pd.DataFrame([], columns = [str(x) for x in m_labels], index=[str(x) for x in m_labels])
        for i in range(len(pair_ws_performance[:,0])):
            if i == (len(pair_ws_performance[:,0])-1):
                M['Alm_PT'][str(pair_ws_performance[i,0])] = nj
            try:
                M[str(pair_ws_performance[i+1,0])][str(pair_ws_performance[i,0])] = input_workstations[-(i+2)]
                if (-(i+1)) == -1:
                    M[str(pair_ws_performance[0,0])]['Alm_MP'] = input_workstations[-(i+1)]

            
            except:
                continue
    else:

    
        while all(isinstance(x, float) for x in input_workstations) == False:

            for idx,  value in enumerate(input_workstations):
                if isinstance(value, str) == True: 
                    if (idx-1) < 0:
                        value_before = nj
                        variable_perf = str(input_workstations[idx])
                    else:
                        value_before = input_workstations[idx-1]
                        variable_perf = str(input_workstations[idx])
                    break

            if variable_perf == 'z2': #must handmade for each mode
                labels_secuence = ['P', '3', 'CC'] #z2 --> 3
                s1_parameters = [0, 0.7, 1, 0.52] #i.e H2O[P,X,H2O/CC,OUTPUT]
                s2_parameters = [1, 0.005, 0, 0.04] #i.e P[P,X,H2O/CC,OUTPUT]
                A = np.array([[1, 1, -1], [s1_parameters[1], s1_parameters[0], -s1_parameters[2]], [s2_parameters[1], s2_parameters[0], -s2_parameters[2]]])
                b = np.array([value_before, value_before*s1_parameters[-1], value_before*s2_parameters[-1]])
                r = np.linalg.solve(A, b)
                value_secuence = [0, r[1], r[2]]

            if variable_perf[0] == '0':
                r = value_before / float(variable_perf)

            for i in range(len(input_workstations)):
                if isinstance(input_workstations[i], str) == True:
                    try:
                        input_workstations[i] = float(r[0])
                        break
                    except:
                        input_workstations[i] = float(r)

            m_labels= set(labels_secuence[::-1] + list(pair_ws_performance[::-1,0]) +['Alm_MP', 'Alm_PT'])


            M = pd.DataFrame([], columns = [str(x) for x in m_labels], index=[str(x) for x in m_labels])

            for i in range(len(pair_ws_performance[:,0])):
                if i == (len(pair_ws_performance[:,0])-1):
                    M['Alm_PT'][str(pair_ws_performance[i,0])] = nj
                try:
                    M[str(pair_ws_performance[i+1,0])][str(pair_ws_performance[i,0])] = input_workstations[-(i+2)]
                    if (-(i+1)) == -1:
                        M[str(pair_ws_performance[0,0])]['Alm_MP'] = input_workstations[-(i+1)]


                except:
                    continue
        #for principal pair

            for i in range(len(labels_secuence)):
                try:
                    M[str(labels_secuence[i+1])][str(labels_secuence[i])] = value_secuence[(i+1)]

        #for other pair

                except:
                    continue
                
    if SUM_M.empty == True:
        SUM_M = SUM_M.reindex_like(M).fillna(0) + M.fillna(0).fillna(0)
    else:
        SUM_M = M.reindex_like(SUM_M).fillna(0) + SUM_M.fillna(0).fillna(0) 
    
    return SUM_M

In [7]:
material_balance(input_ws, pair_ws_perf)

,2,P,Alm_MP,3,Alm_PT,4,CC,1
2,0.000000,0.0,0.0,1521.964718,0.0,0.000000,0.000000,0.000000
P,0.000000,0.0,0.0,32.542373,0.0,0.000000,0.000000,0.000000
Alm_MP,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,1521.964718
3,0.000000,0.0,0.0,0.000000,0.0,1491.525424,524.067797,0.000000
Alm_PT,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
4,0.000000,0.0,0.0,0.000000,1000.0,0.000000,0.000000,0.000000
CC,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
1,1521.964718,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000


As shown previously, the matrix is computed by flows in kg, corresponding to flow in the potatoes chip process, it's completely possible to input other products to fill out the matrix as long as products have the facility take on regard, moreover, the table would used as data entry in models like triangle method developed by Bloch and improved by Schmigalla [show related article](https://www.researchgate.net/publication/319678187_Designing_a_Layout_Using_Schmigalla_Method_Combined_with_Software_Tool_visTABLE). , covered up in other of my GitHub Repos